## Setup

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!export PYTHONPATH=$PYTHONPATH:"drive/MyDrive/OML_project"

In [3]:
%cd "gdrive/MyDrive/OML_project"

/content/gdrive/.shortcut-targets-by-id/15pSdFqg0ADOSoj1lZ5AbPzSZuRdumG6a/OML_project


In [15]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jun  7 11:58:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [11]:
cd /content/gdrive/MyDrive/OML_project/

/content/gdrive/.shortcut-targets-by-id/15pSdFqg0ADOSoj1lZ5AbPzSZuRdumG6a/OML_project


In [12]:
!git init

Initialized empty Git repository in /content/gdrive/.shortcut-targets-by-id/15pSdFqg0ADOSoj1lZ5AbPzSZuRdumG6a/OML_project/.git/


In [20]:
!git add .


In [21]:
!git commit -m "Updated configs"

[master 96f7865] Updated configs
 1 file changed, 1 insertion(+), 1 deletion(-)


In [22]:
!git config --global user.name "arinaraileanu"

In [23]:
!git remote add origin https://github.com/MihaiDavid05/OML_Project

In [27]:
!git remote set-url origin git@github.com:MihaiDavid05/OML_Project.git

In [29]:
!git push --set-upstream origin master

Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


## Training

Change only **config_filename** variable!

In [17]:
import argparse
import os
import json
import torch
from utils.network import build_network
from utils.dataset import build_dataset
from utils.train import train
from utils.utils import setup
from torch.utils.tensorboard import SummaryWriter


# CHANGE ONLY THIS PARAMETER
config_filename = "config2"

# Create configs, logs and checkpoints folder
if not os.path.exists("configs/"):
    os.mkdir("configs/")
if not os.path.exists("logs/"):
    os.mkdir("logs/")
if not os.path.exists("checkpoints/"):
    os.mkdir("checkpoints/")

config_path = 'configs/' + config_filename + '.json'
# Get config
with open(config_path) as json_config:
    config = json.load(json_config)

config["name"] = config_filename

# Set folder for logging
log_dir = config["log_dir"] + '/' + config["name"]
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

# Set folder for checkpoints
checkpoints_dir = config["checkpoint_dir"] + '/' + config["name"]
if not os.path.exists(checkpoints_dir):
    os.mkdir(checkpoints_dir)

config["checkpoint_dir"] = checkpoints_dir
config["log_dir"] = log_dir

# Check for cuda availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device state: ', device)

# Get dataset
dataset = build_dataset(config, config["data"], train=True)

# Get network
net = build_network(config)
net.to(device=device)

# Train network
writer = SummaryWriter(log_dir=config["log_dir"])
train(dataset, net, config, writer, device=device)


FileNotFoundError: ignored

## Use Tensorboard

Change log folder (logs/**\<change_this\>**) to the desired one!

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=logs/config2

## Event parser

Change only **log_config_name** variable!

In [ ]:
import argparse
from utils.utils import get_events_data


log_config_name = "config2"

log_path = f"logs/{log_config_name}"
get_events_data(log_path, False, True, log_path, to_watch='_lr')
get_events_data(log_path + "/Loss_train", False, True, log_path, to_watch='_loss_train')
get_events_data(log_path + "/Loss_val", False, True, log_path, to_watch='_loss_val')
get_events_data(log_path + "/Accuracy_train", False, True, log_path, to_watch='_acc_train')
get_events_data(log_path + "/Accuracy_val", False, True, log_path, to_watch='_acc_val')

## Testing

Change only **config_filename** variable! Also, you can change **image_indexes** to a specific list of indexes if you want to test only some images.

In [ ]:
import os
import json
import torch
from utils.train import predict
from utils.network import build_network
from utils.dataset import build_dataset

# CHANGE ONLY THIS PARAMETER
config_filename = "config2"
image_indexes = None

# Create configs, logs and checkpoints folder
if not os.path.exists("configs/"):
    os.mkdir("configs/")
if not os.path.exists("logs/"):
    os.mkdir("logs/")
if not os.path.exists("checkpoints/"):
    os.mkdir("checkpoints/")

config_path = 'configs/' + config_filename + '.json'
# Get config
with open(config_path) as json_config:
    config = json.load(json_config)

config["name"] = config_filename

# Set folder for logging
log_dir = config["log_dir"] + '/' + config["name"]
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

# Set folder for checkpoints
checkpoints_dir = config["checkpoint_dir"] + '/' + config["name"]
if not os.path.exists(checkpoints_dir):
    os.mkdir(checkpoints_dir)

config["checkpoint_dir"] = checkpoints_dir
config["log_dir"] = log_dir

# Check for cuda availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device state: ', device)

# Get network
net = build_network(config)
net.to(device=device)
model_path = config["checkpoint_dir"] + '/' + 'bestmodel.pth'
net.load_state_dict(torch.load(model_path, map_location=device))

# Get an image
test_dataset = build_dataset(config, config["data"])

# Make predictions
predict(test_dataset, net, device, image_indexes)

In [ ]:
!python3 experiments.py